# Prepare Open AI client object

In [1]:
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
client

# Prepare documents for information retrieval

In [6]:
import json

In [7]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

# RAG with Elastic Search

In [9]:
from elasticsearch import Elasticsearch

In [10]:
es_client = Elasticsearch("http://localhost:9200")

In [11]:
# es_client.info()

In [12]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-questions"

# Create an index in elastic search (equivelant to a table in sql database)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
for doc in documents:

    es_client.index(index=index_name, document=doc)

In [14]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        # a question is 3 times more important than the text or section for the search results
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields",
                    }
                },
                "filter": {  # limit questions about only one part of the documents available
                    "term": {"course": "data-engineering-zoomcamp"}
                },
            }
        },
    }

    search_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in search_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [15]:
def build_prompt(query, search_results):

    prompt_template = """
    QUESTION: {question}

    CONTEXT:
    {context}
    
    ANSWER:
    """.strip()

    context_template = """
        section:{section}
        question: {question}
        answer: {text}
        """.strip()

    context_list = []

    for doc in search_results:
        context_list.append(context_template.format(**doc))

    context = "/n/n".join(context_list)

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

# Load open source model from Hugging Face AI

Explanation of Parameters:

- max_length: Set this to a higher value if you want longer responses. For example, max_length=300.
num_beams: Increasing this can lead to more thorough exploration of possible sequences. Typical values are between 5 and 10.
- do_sample: Set this to True to use sampling methods. This can produce more diverse responses.
temperature: Lowering this value makes the model more confident and deterministic, while higher values increase diversity. Typical values range from 0.7 to 1.5.
- top_k and top_p: These parameters control nucleus sampling. top_k limits the sampling pool to the top k tokens, while top_p uses cumulative probability to cut off the sampling pool. Adjust these based on the desired level of randomness.

## microsoft/Phi-3-mini-128k-instruct

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [20]:
def llm(prompt):

    messages = [
        {"role": "user", "content": prompt},
    ]

    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)

    return output[0]["generated_text"]

In [21]:
def elastic_rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return search_results, prompt, answer

In [22]:
query = "i just found out about this course, can i still join?"

In [23]:
search_results, prompt, answer = elastic_rag(query)

In [24]:
answer

" Yes, you can still join the course even if you find out about it after the start date. You are still eligible to submit the homeworks, but be aware that there will be deadlines for turning in the final projects. It's recommended not to leave everything for the last minute."